In [2]:
import anthropic
import pandas as pd
import ast
import re

In [3]:
def get_api_key(file_path):
    with open(file_path, "r") as file:
        return file.read().strip()

In [4]:
def extract_valid_list(response_text):
    
    response_text = re.sub(r"```[a-zA-Z]*\n", "", response_text)  
    response_text = response_text.strip("`")  

    match = re.search(r"\[[^\]]+\]", response_text, re.DOTALL)  

    if match:
        list_text = match.group(0)
        try:
            extracted_list = ast.literal_eval(list_text)  
            if isinstance(extracted_list, list):
                return extracted_list
        except Exception as e:
            print(f"Error parsing response: {e}")  
            return []  
    
    return []

In [5]:
def generate_ner_tags(sentence, client):
    system_prompt = (
        "You are an AI language model that outputs only valid Python lists. "
        "No explanations. No additional text. No Markdown formatting."
    )

    user_prompt = f"""
    You are a highly intelligent AI trained for named entity recognition.
    Your task is to assign the most suitable ner_tags to every token in the following text based on the provided list of possible ner_tags.
    
    Assign NER tags from the set:
    ['B-Further Specification', 'I-Further Specification', 'O', 'I-XOR Gateway',
     'B-XOR Gateway', 'B-Activity Data', 'B-Condition Specification', 'I-Actor',
     'B-AND Gateway', 'I-Activity','B-Activity', 'I-Activity Data',
     'I-Condition Specification', 'I-AND Gateway', 'B-Actor']
    to each token in:
    {sentence}

    Output ONLY a Python-style list of NER tags, e.g.:
    ['O', 'O', 'B-Actor', 'I-Actor']

    No explanations. No additional text. Do not include ```python or any Markdown formatting.
    """

    response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1024,
        temperature=0.0,
        system=system_prompt,
        messages=[{"role": "user", "content": user_prompt}]
    )

    if not response.content or not response.content[0].text:
        print("DEBUG: Empty response from Claude API")
        return []

    response_text = response.content[0].text.strip()
    print(f"DEBUG: Raw API Response: {response_text}") 

    return extract_valid_list(response_text)

In [6]:
def process_dataset(input_file, output_file, api_key_file):

    api_key = get_api_key(api_key_file)
    
    client = anthropic.Anthropic(api_key=api_key)  

    dataset = pd.read_csv(input_file)

    def process_row(row):
        try:
            tokens = ast.literal_eval(row["tokens"]) 
            sentence = " ".join(tokens)
            return generate_ner_tags(sentence, client)
        except Exception as e:
            return f"Error: {str(e)}"

    dataset["claude-3-haiku-latest_ner_tags"] = dataset.apply(process_row, axis=1)

    dataset.to_csv(output_file, index=False)
    print(f"Updated dataset saved to {output_file}")

In [7]:
input_file = "PET-Customized.csv" 
output_file = "PET-Customized-Claude3.0-Haiku.csv" 
api_key_file = "./API_Key/Anthropic.txt"

if __name__ == "__main__":
    process_dataset(input_file, output_file, api_key_file)

DEBUG: Raw API Response: ['B-Actor', 'O', 'B-Activity', 'O']
DEBUG: Raw API Response: ['B-Actor', 'I-Actor', 'O', 'B-Activity', 'I-Activity', 'O', 'B-Activity Data', 'O', 'O']
DEBUG: Raw API Response: ['B-Actor', 'O', 'O', 'O', 'O', 'O', 'B-Activity', 'I-Activity', 'O', 'O', 'O', 'O', 'O']
DEBUG: Raw API Response: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
DEBUG: Raw API Response: ['O', 'O', 'B-Actor', 'I-Actor', 'O', 'B-Activity', 'I-Activity', 'O']
DEBUG: Raw API Response: ['B-Activity', 'O', 'B-Activity Data', 'O', 'B-Activity', 'O', 'B-Activity Data', 'O', 'B-Activity', 'O', 'B-Activity Data', 'O']
DEBUG: Raw API Response: ['O', 'O', 'B-Activity Data', 'I-Activity Data', 'O', 'B-Activity', 'I-Activity', 'O']
DEBUG: Raw API Response: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
DEBUG: Raw API Response: ['O', 'O', 'B-Activity', 'I-Activity', 'O', 'B-Further Specification', 'I-Further Specification', 'O', 'B-Activity Data', 'I-Activity Data']
DEBUG: Raw API Response: ['O', 'O', 'O', 'O', 'O